In [69]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import os
import re

In [70]:
def save_novel():
    count=0

    for i in range(200):


        soup = BeautifulSoup(driver.page_source, 'html.parser')
        end_point=soup.find('span',class_='content-wrap')#後續為付費內容，無法讀取全文

        if  '上架感言' in end_point.text:#總共多開了1個分頁，不用時將之關閉
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            break

        useless_chapter=soup.find('span',class_='content-wrap')
        
        if re.search('第[\w]*章', useless_chapter.text)==None and i!=0:#中間內容可能有請假，非章節內容
            print(useless_chapter.text)
            driver.find_element(By.PARTIAL_LINK_TEXT, '下一章').click()
            

            time.sleep(3)
            pass
        count+=1
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        if i==0:
            title=soup.find('h1').text#小說名稱
        thumb_up=soup.find('span',class_='j_admireNum')#讚賞人數
        result=soup.find_all('span',class_='content-wrap')[1:]#內文

        string=''
        for i in range(len(result)):
            string+=result[i].text.lstrip()
        string+='?????'+thumb_up.text

        if not os.path.isdir('novel/'+title):
            os.mkdir('novel/'+title)
        path = 'novel/'+title+'/'+str(count)+'.txt'
        f = open(path, 'w', encoding='UTF-8')
        f.write(string)
        f.close()

        driver.find_element(By.PARTIAL_LINK_TEXT, '下一章').click()

        time.sleep(3)

In [71]:
options = Options()
options.add_argument("--disable-notifications")

# 啟動 chrome driver
 
driver = webdriver.Chrome('./chromedriver', options=options)
# 指定開啟網址
driver.get("https://www.qidian.com/rank/yuepiao/") 
button= WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME,'rank-body'))                                                          
        )
novel=button.find_elements(By.TAG_NAME,'li')

flag=0
for i in range(2,5,1):
    novel[i].find_element(By.TAG_NAME,'a').click()
    driver.switch_to.window(driver.window_handles[1])
    
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="readBtn"]'))                                                          
    ).click()#start reading
    
    if flag==0:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.LINK_TEXT,'关闭'))                                                          
        ).click()#when first visit,need to close advertise
        flag=1
    save_novel()

7月结束，8月开干
请假一章
关于红月这本书！


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=96.0.4664.45)


In [ ]:
driver.close()